# Урок 6. Типы медицинских данных
## Домашнее задание.

John Snow Labs предоставляют простой и унифицированный API Python для обработки текстовых данных с помощью NLP на профессиональном уровне.

Изучите одно из предлагаемых решений процессинга клинических данных по поиску именованных сущностей (NER).
https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/1.Clinical_Named_Entity_Recognition_Model.ipynb

Протестируйте распознавание моделью терминов на тексте из какой-либо статьи раздела Case Reports в PubMed NCBI.


In [2]:
import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

Saving spark_nlp_for_healthcare_spark_ocr_8489.json to spark_nlp_for_healthcare_spark_ocr_8489.json


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!chcp 65001

/bin/bash: line 1: chcp: command not found


In [6]:
os.listdir('/content/drive/MyDrive/Colab Notebooks/GB/20_6_files')

['spark_nlp_for_healthcare_spark_ocr_8489.json',
 'cureus-0015-00000037851.pdf',
 'cureus-0015-00000037851.txt']

In [5]:
license_keys.keys()

dict_keys(['SPARK_OCR_LICENSE', 'SPARK_OCR_SECRET', 'OCR_VERSION', 'SPARK_NLP_LICENSE', 'SECRET', 'JSL_VERSION', 'PUBLIC_VERSION', 'AWS_ACCESS_KEY_ID', 'AWS_SECRET_ACCESS_KEY', 'AWS_SESSION_TOKEN'])

In [8]:
license_keys['JSL_VERSION']

'5.1.2'

In [9]:
license_keys['PUBLIC_VERSION']

'5.1.2'

In [10]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.1 spark-nlp==$PUBLIC_VERSION

# Installing NLU
! pip install --upgrade --q nlu==4.0.1rc4 --no-dependencies

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.3/536.3 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.6/570.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.2/464.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.0 MB/s eta 0:00:00


### Настройка сессии Spark

In [11]:
import sparknlp
import sparknlp_jsl
import nlu

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

Spark NLP Version : 5.1.2
Spark NLP_JSL Version : 5.1.2


In [12]:
# if you want to start the session with custom params as in start function above
from pyspark.sql import SparkSession

def start(SECRET):
    builder = SparkSession.builder \
        .appName("Spark NLP Licensed") \
        .master("local[*]") \
        .config("spark.driver.memory", "16G") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.kryoserializer.buffer.max", "2000M") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:"+PUBLIC_VERSION) \
        .config("spark.jars", "https://pypi.johnsnowlabs.com/"+SECRET+"/spark-nlp-jsl-"+JSL_VERSION+".jar")

    return builder.getOrCreate()

#spark = start(SECRET)

In [7]:
# with open("cureus-0015-00000037851.txt", "r") as file:
with open("/content/drive/MyDrive/Colab Notebooks/GB/20_6_files/cureus-0015-00000037851.txt", "r") as file:
    report = file.readlines()

In [13]:
report_text = ''.join(report).replace('\n', '').replace('\t', '')
report_text

"Open Access Case Report DOI: 10.7759/cureus.37851 Non-small Cell Lung Carcinoma With Metastases to Various Levels of Vertebra: A Case Report and Literature Review  Review began 03/27/2023  Review ended 04/16/2023  Published 04/19/2023 © Copyright 2023 Inban et al. This is an open access article distributed under the terms of the Creative Commons Attribution License CC-BY 4.0., which permits unrestricted use, distribution, and reproduction in any medium, provided the original author and source are credited.  Pugazhendi Inban   , Tamer Zahdeh   , Sai Harini Chandrasekaran   , Tarsha A. Intsiful   , Chengala Ananyaa Gowthavaram   , Abasiono K. Ebong   , Yabets Kejela   , Satyam Singh   , Aadil Khan   , Ayushi Awale  1. Department of General Medicine, Government Medical College, Omandurar, Chennai, IND  2. Internal Medicine, Hadassah Medical Center, Jerusalem, ISR 3. College of Medicine, University of Ghana Medical Center, Accra, GHA  4. Internal Medicine, Malla Reddy Institute of Medical

### Clinical NER Pipeline (with pretrained models)

In [15]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_clinical_large","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner")\
    .setLabelCasing("upper") #decide if we want to return the tags in upper or lower case

ner_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner"])\
    .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(
    stages=[
        documentAssembler,
        sentenceDetector,
        tokenizer,
        word_embeddings,
        clinical_ner,
        ner_converter
        ])


empty_data = spark.createDataFrame([[""]]).toDF("text")

model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical_large download started this may take some time.
[OK!]


In [16]:
model.stages

[DocumentAssembler_6463f963cbfe,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_dc5bf7d0dfdf,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_1a8637089929,
 NER_CONVERTER_4ee3e4a59ca9]

In [17]:
clinical_ner.getClasses()

['O',
 'B-TREATMENT',
 'I-TREATMENT',
 'B-PROBLEM',
 'I-PROBLEM',
 'B-TEST',
 'I-TEST']

In [18]:
clinical_ner.extractParamMap()

{Param(parent='MedicalNerModel_1a8637089929', name='includeAllConfidenceScores', doc='whether to include all confidence scores in annotation metadata or just the score of the predicted tag'): False,
 Param(parent='MedicalNerModel_1a8637089929', name='inferenceBatchSize', doc='number of sentences to process in a single batch during inference'): 1,
 Param(parent='MedicalNerModel_1a8637089929', name='labelCasing', doc='Setting all labels of the NER models upper/lower case. values upper|lower'): 'upper',
 Param(parent='MedicalNerModel_1a8637089929', name='sentenceTokenIndex', doc='Whether to include the token index for each sentence in annotation metadata.'): False,
 Param(parent='MedicalNerModel_1a8637089929', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='MedicalNerModel_1a8637089929', name='includeConfidence', doc='whether to include confidence scores in annotation metadata'): True,
 Param(parent='MedicalNerModel_1a8637

In [19]:
clinical_ner.getStorageRef()

'clinical'

### LightPipelines

In [20]:
# fullAnnotate in LightPipeline

# text = '''
# A 28-year-old female with a history of gestational diabetes mellitus diagnosed eight years prior to presentation and subsequent type two diabetes mellitus (T2DM), one prior episode of HTG-induced pancreatitis three years prior to presentation, and associated with an acute hepatitis, presented with a one-week history of polyuria, poor appetite, and vomiting.
# She was on metformin, glipizide, and dapagliflozin for T2DM and atorvastatin and gemfibrozil for HTG. She had been on dapagliflozin for six months at the time of presentation.
# Physical examination on presentation was significant for dry oral mucosa ; significantly , her abdominal examination was benign with no tenderness, guarding, or rigidity. Pertinent laboratory findings on admission were: serum glucose 111 mg/dl,  creatinine 0.4 mg/dL, triglycerides 508 mg/dL, total cholesterol 122 mg/dL, and venous pH 7.27.
# '''

text = report_text

print (text)

light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)


chunks = []
entities = []
sentence= []
begin = []
end = []
confidence = []

for n in light_result[0]['ner_chunk']:

    begin.append(n.begin)
    end.append(n.end)
    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    sentence.append(n.metadata['sentence'])
    confidence.append(n.metadata["confidence"])


df_clinical = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end,
                   'sentence_id':sentence, 'entities':entities, 'confidence':confidence})

df_clinical.head(20)

Open Access Case Report DOI: 10.7759/cureus.37851 Non-small Cell Lung Carcinoma With Metastases to Various Levels of Vertebra: A Case Report and Literature Review  Review began 03/27/2023  Review ended 04/16/2023  Published 04/19/2023 © Copyright 2023 Inban et al. This is an open access article distributed under the terms of the Creative Commons Attribution License CC-BY 4.0., which permits unrestricted use, distribution, and reproduction in any medium, provided the original author and source are credited.  Pugazhendi Inban   , Tamer Zahdeh   , Sai Harini Chandrasekaran   , Tarsha A. Intsiful   , Chengala Ananyaa Gowthavaram   , Abasiono K. Ebong   , Yabets Kejela   , Satyam Singh   , Aadil Khan   , Ayushi Awale  1. Department of General Medicine, Government Medical College, Omandurar, Chennai, IND  2. Internal Medicine, Hadassah Medical Center, Jerusalem, ISR 3. College of Medicine, University of Ghana Medical Center, Accra, GHA  4. Internal Medicine, Malla Reddy Institute of Medical 

,chunks,begin,end,sentence_id,entities,confidence
0,Non-small Cell Lung Carcinoma,50,78,1,PROBLEM,0.91742504
1,Metastases,85,94,1,PROBLEM,0.9984
2,the Creative Commons Attribution License CC-BY,327,372,1,TREATMENT,0.7040667
3,Pugazhendi Inban,513,528,2,PROBLEM,0.5691
4,Intsiful,591,598,3,PROBLEM,0.9823
5,Ayushi,709,714,3,PROBLEM,0.9762
6,lung cancer,1494,1504,13,PROBLEM,0.8838
7,quite deadly,1538,1549,13,PROBLEM,0.71924996
8,its brutal mortality rate,1612,1636,14,PROBLEM,0.63235
9,the comorbidities,1639,1655,14,PROBLEM,0.98475003


In [21]:
light_result[0]

{'document': [Annotation(document, 0, 16106, Open Access Case Report DOI: 10.7759/cureus.37851 Non-small Cell Lung Carcinoma With Metastases to Various Levels of Vertebra: A Case Report and Literature Review  Review began 03/27/2023  Review ended 04/16/2023  Published 04/19/2023 © Copyright 2023 Inban et al. This is an open access article distributed under the terms of the Creative Commons Attribution License CC-BY 4.0., which permits unrestricted use, distribution, and reproduction in any medium, provided the original author and source are credited.  Pugazhendi Inban   , Tamer Zahdeh   , Sai Harini Chandrasekaran   , Tarsha A. Intsiful   , Chengala Ananyaa Gowthavaram   , Abasiono K. Ebong   , Yabets Kejela   , Satyam Singh   , Aadil Khan   , Ayushi Awale  1. Department of General Medicine, Government Medical College, Omandurar, Chennai, IND  2. Internal Medicine, Hadassah Medical Center, Jerusalem, ISR 3. College of Medicine, University of Ghana Medical Center, Accra, GHA  4. Interna

In [22]:
light_model_with_emb = LightPipeline(model, parse_embeddings=True)
light_result_with_emb = light_model_with_emb.annotate(text)

list(zip(light_result_with_emb['token'], light_result_with_emb['embeddings']))[:3]

[('Open',
  '0.12604491 -0.44639736 0.14473958 -0.13526338 -0.3160966 -0.12418029 0.45730028 0.082356006 -0.08205182 0.14022872 -0.3375427 0.30959594 0.24423097 -4.6692794E-4 -0.21215896 0.080081254 -0.016052276 -0.041487444 0.10208524 0.51103866 -0.5625145 -0.7215034 0.35992652 0.76234233 -0.21661724 -0.10605395 -1.2641022 -0.36224657 0.16525728 -0.15575317 -0.44869855 -0.48575163 0.61529106 0.23247272 0.22499801 -0.4517376 0.3304193 0.32444906 0.21291961 -0.24843968 -0.41587958 -0.08342149 -0.22666891 -0.63868415 0.07066457 -0.7487918 -0.110895455 -0.30577308 0.30167633 0.08464174 0.760614 -0.34753948 0.103562854 0.4400293 -0.48112723 -0.24779673 0.091330454 -0.12671559 0.5498055 -0.3489189 0.031242583 -0.0899303 0.011820456 0.15180753 -0.90213007 0.13662997 0.44735336 -0.4280147 -0.19219898 0.49763548 -0.007193502 0.045628108 -0.36349982 -0.08470682 0.3506332 -0.040930312 -0.104178466 -0.21209595 -0.73254746 0.4842284 0.20349182 0.2814869 -0.47110233 -0.13505551 -0.40281662 -0.12103

**Prettifying the results with NLU**  

John Snow Labs' NLU is a Python library for applying state-of-the-art text mining, directly on any dataframe, with a single line of code. We will use some functionalities of NLU to prettify our results.

In [23]:
nlu.to_pretty_df(model, text, positions=True, output_level='chunk').columns

Index(['document', 'document_begin', 'document_end', 'entities_ner_chunk',
       'entities_ner_chunk_begin', 'entities_ner_chunk_class',
       'entities_ner_chunk_confidence', 'entities_ner_chunk_end',
       'entities_ner_chunk_origin_chunk', 'entities_ner_chunk_origin_sentence',
       'ner_iob_ner_begin', 'ner_iob_ner_end', 'sentence_dl',
       'sentence_dl_begin', 'sentence_dl_end', 'token_begin', 'token_end',
       'word_embedding_embeddings', 'word_embedding_embeddings_begin',
       'word_embedding_embeddings_end'],
      dtype='object')

In [24]:
#For a given pipeline output level is automatically set to the last anntator's output level by default.
#This can be changed by defining the value of "output_level" parameter.
#Values can be set as; token, sentence, document, chunk and relation .

cols = [
     'entities_ner_chunk',
     'entities_ner_chunk_begin',
     'entities_ner_chunk_end',
     'entities_ner_chunk_origin_sentence',
     'entities_ner_chunk_class',
]
df_clinical = nlu.to_pretty_df(model, text, positions=True, output_level='chunk')[cols]
df_clinical.head(20)

,entities_ner_chunk,entities_ner_chunk_begin,entities_ner_chunk_end,entities_ner_chunk_origin_sentence,entities_ner_chunk_class
0,Non-small Cell Lung Carcinoma,50,78,1,PROBLEM
0,Metastases,85,94,1,PROBLEM
0,the Creative Commons Attribution License CC-BY,327,372,1,TREATMENT
0,Pugazhendi Inban,513,528,2,PROBLEM
0,Intsiful,591,598,3,PROBLEM
0,Ayushi,709,714,3,PROBLEM
0,lung cancer,1494,1504,13,PROBLEM
0,quite deadly,1538,1549,13,PROBLEM
0,its brutal mortality rate,1612,1636,14,PROBLEM
0,the comorbidities,1639,1655,14,PROBLEM


### NER Visualizer

For saving the visualization result as html, provide `save_path` parameter in the display function.

In [27]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

visualiser.display(light_result[0], label_col='ner_chunk', document_col='document', save_path="display_result.html")


# Change color of an entity label

# visualiser.set_label_colors({'PROBLEM':'#008080', 'TEST':'#800080', 'TREATMENT':'#808080'})
# visualiser.display(light_result[0], label_col='ner_chunk')


# Set label filter

# visualiser.display(light_result[0], label_col='ner_chunk', document_col='document',
                   #labels=['PROBLEM','TEST'])

### ModelTracer  

`ModelTracer` allows to track the uids and timestamps of each stage of the pipeline.

Let's apply it over an empty data with our existing pipeline.

In [33]:
from sparknlp_jsl.modelTracer import ModelTracer

data = spark.createDataFrame([['']]).toDF("text")

df = nlpPipeline.fit(data).transform(data)

tracer_result = ModelTracer().addUidCols(pipeline = nlpPipeline, df = df)

tracer_result.show(truncate=False)

+----+------------------------------------------+--------+-----+----------+---+---------+----------------------------------------------------------------------+----------------------------------------------------------------------------+--------------------------------------------------------------+--------------------------------------------------------------------------+--------------------------------------------------------------------+-------------------------------------------------------------------------+
|text|document                                  |sentence|token|embeddings|ner|ner_chunk|documentassembler_model_uid                                           |sentencedetectordlmodel_model_uid                                           |tokenizer_model_uid                                           |word_embeddings_model_model_uid                                           |medicalnermodel_model_uid                                           |nerconverterinternal_model_uid       

### Output widgets

In [32]:
from google.colab import widgets

t = widgets.TabBar(["ner_clinical_large", "ner_jsl", "viz_clinical", "viz_jsl"])

with t.output_to(0):
    display(df_clinical)

# with t.output_to(1):
#     display(jsl_df)

with t.output_to(2):
    visualiser.display(light_result[0], label_col='ner_chunk', document_col='document')

# with t.output_to(3):
#     visualiser.display(jsl_light_result[0], label_col='jsl_ner_chunk', document_col='document')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

__*Комментарии преподавателя:*__<br>
*Дарья Бородко・Преподаватель<br>*

>*Добрый день!  
...*